In [ ]:
%pylab inline
%config InlineBackend.figure_formats = ['retina']

import pandas as pd
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Feature Engineering with Linear Regression: Applied to the Ames Housing Data

Using the Ames Housing Data:

Dean De Cock
Truman State University
Journal of Statistics Education Volume 19, Number 3(2011), www.amstat.org/publications/jse/v19n3/decock.pdf

In this notebook, we will build some linear regression models to predict housing prices from this data. In particular, we will set out to improve on a baseline set of features via **feature engineering**: deriving new features from our existing data. Feature engineering is done extensively in machine learning applications, and it often makes the difference between a weak or decent model and a strong one. Sometimes feature engineering is even more important than model selection and hyperparameter tuning when it comes to improving predictions!      

We will split our data into training and validation sets, build models on various feature sets and compare their results on the validation set. We will use visual exploration, domain understanding, and intuition to construct new features that add significant predictive signal, which will be apparent in validation *r-squared* scores.

**Notebook Contents**

> 1. Simple EDA and baseline model
> 2. Basic feature engineering: adding polynomial terms
> 3. Basic feature engineering: adding interaction terms
> 4. Intermediate feature engineering: categories and features derived from category aggregates 

## 1. Simple EDA and Baseline Model

#### Load the Data, Examine and Explore

In [ ]:
## Load in the Ames Housing Data
datafile = "data/Ames_Housing_Data.tsv"
df=pd.read_csv(datafile, sep='\t')

In [ ]:
## Examine the columns, look at missing data
df.info()

In [ ]:
# This is recommended by the data set author to remove a few outliers

df = df.loc[df['Gr Liv Area'] <= 4000,:]
df.shape

There are a *lot* of variables, many of which have a lot of missing values.  Let's pick out just a few columns and start building models using that.

In [ ]:
smaller_df= df.loc[:,['Lot Area', 'Overall Qual', 'Overall Cond', 
                      'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
                      'Full Bath', 'Bedroom AbvGr', 'Fireplaces', 
                      'Garage Cars','SalePrice']]

In [ ]:
smaller_df.describe()

In [ ]:
smaller_df.info()

In [ ]:
# There appears to be one NA in Garage Cars - fill with 0
smaller_df = smaller_df.fillna(0)

In [ ]:
smaller_df.info()

Now that we have a nice, filtered dataset, let's generate visuals to better understand the target and feature-target relationships: pairplot is great for this!

In [ ]:
sns.pairplot(smaller_df[:1000], plot_kws=dict(alpha=.1, edgecolor='none'))

---
**Data Exploration Exercises**: 

1. What do these plots tell us about the distribution of the target?   

2. What do these plots tell us about the relationship between the features and the target? Do you think that linear regression is well-suited to this problem? Do any feature transformations come to mind?

3. What do these plots tell us about the relationship between various pairs of features? Do you think there may be any problems here? 

---

#### Setting up for modeling and building a baseline:

In [ ]:
#Separate our features from our target

X = smaller_df.loc[:,['Lot Area', 'Overall Qual', 'Overall Cond', 
                      'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
                      'Full Bath', 'Bedroom AbvGr', 'Fireplaces', 
                      'Garage Cars']]

y = smaller_df['SalePrice']

In [ ]:
X.info()

Now that we have feature/target data X, y ready to go, we're nearly ready to fit and evaluate a baseline model using our current feature set. We'll need to create a **train/validation split** before we fit and score the model. 

Since we'll be repeatedly splitting X, y into the same train/val partitions and fitting/scoring new models as we update our feature set, we'll define a reusable function that completes all these steps, making our code/process more efficient going forward. 

In [ ]:
def split_and_validate(X, y):
    '''
    For a set of features and target X, y, perform a 80/20 train/val split, 
    fit and validate a linear regression model, and report results
    '''
    
    # perform train/val split
    X_train, X_val, y_train, y_val = \
        train_test_split(X, y, test_size=0.2, random_state=42)
    
    # fit linear regression to training data
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    
    # score fit model on validation data
    val_score = lr_model.score(X_val, y_val)
    
    # report results
    print('\nValidation R^2 score was:', val_score)
    print('Feature coefficient results: \n')
    for feature, coef in zip(X.columns, lr_model.coef_):
        print(feature, ':', f'{coef:.2f}') 
        

Great, let's go ahead and run this function on our baseline feature set and take some time to analyze the results.

In [ ]:
split_and_validate(X, y)

-----
**Review Exercise**: How is the R^2 score defined? What does this score mean intuitively? What are possible drawbacks of using the R^2 score for evaluation? 

-----

## 2. Basic feature engineering: adding polynomial terms

One of the first things that we looked for in the pairplot was evidence about the relationship between each feature and the target. In certain features like _'Overall Qual'_ and _'Gr Liv Qual'_, we notice an upward-curved relationship rather than a simple linear correspondence. This suggests that we should add quadratic **polynomial terms or transformations** for those features, allowing us to express that non-linear relationship while still using linear regression as our model.

Luckily, pandas makes it quite easy to quickly add those square terms as additional features to our original feature set. We'll do so and evaluate our model again below.

As we add to our baseline set of features, we'll create a copy of the latest benchmark so that we can continue to store our older feature sets. **Note that you should be very careful about this in practice**, as it means that we are saving several redundant copies of the data: it's often better to continuously override the original feature set in order to save RAM.

In [ ]:
X2 = X.copy()

X2['OQ2'] = X2['Overall Qual'] ** 2
X2['GLA2'] = X2['Gr Liv Area'] ** 2

split_and_validate(X2, y)

As we might expect, adding appropriate square terms allows our model to do a significantly better job (+.05 R^2) capturing certain feature-target relationships that are closer to quadratic than linear. If we saw higher-order curve relationships, we could try adding higher degree polynomial terms as well.

**Note**: feature transformations are not limited to polynomial terms and can also include log and square root transforms among others. Follow your instinct based on what you see in feature-target plots, and validate!

-----
**Polynomial Feature Exercise**: Based on inspecting the pairplot, what other features do you think polynomial terms may be helpful for? Try adding them to X2 and rerunning `split_and_validate`. Is the improvement less than you expected? What do you think happened?  

-----

## 3. Basic feature engineering: adding interaction terms

With our current feature set, each feature value has no influence over how the model views other features' values. Each feature is treated as a completely independent quantity. However, there may easily be **interaction effects** present, in which the impact of one feature may dependent on the current value of a different feature.

For example, there may be a higher premium for increasing _'Overall Qual'_ for houses that were built more recently. If such a premium or a similar effect exists, a feature that multiplies _'Overall Qual'_ by _'Year Built'_ can help us capture it.

Another style of interaction term involves feature proprtions: for example, to get at something like quality per square foot we could divide _'Overall Qual'_ by _'Lot Area'_.

Let's try adding both of these interaction terms and see how they impact the model results.

In [ ]:
X3 = X2.copy()

# multiplicative interaction
X3['OQ_x_YB'] = X3['Overall Qual'] * X3['Year Built']

# division interaction
X3['OQ_/_LA'] = X3['Overall Qual'] / X3['Lot Area']

split_and_validate(X3, y)

Great, they gave us an additional boost of .01 R^2.

-----
**Interaction Feature Exercise**: What other interactions do you think might be helpful? Try adding them to X3 and measuring your results!  

-----

## 4. Intermediate feature engineering: categories and features derived from category aggregates 

Incorporating **categorical features** into linear regression models is fairly straightforward: we can create a new feature column for each category value, and fill these columns with 1s and 0s to indicate which category is present for each row. This method is called **dummy variables** or **one-hot-encoding**.

We'll first explore this using the _'House Style'_ feature from the original dataframe. Before going straight to dummy variables, it's a good idea to check category counts to make sure all categories have reasonable representation.

In [ ]:
df['House Style'].value_counts()

This looks ok, and here's a quick look at how dummy features actually appear:

In [ ]:
pd.get_dummies(df['House Style']).head()

We can call `pd.get_dummies()` on our entire dataset to quickly get data with all the original features and dummy variable representation of any categorical features. Let's do that below and run a new benchmark!

In [ ]:
X4 = X3.copy()

X4['House Style'] = df['House Style']

split_and_validate(pd.get_dummies(X4), y)

Cool, we gained another .01 R^2. Let's think about bringing in another categorical variable, _Neighborhood_, from the original data.

In [ ]:
nbh_counts = df.Neighborhood.value_counts()
nbh_counts

For this category, let's map the few least-represented neighborhoods to an "other" category before adding the feature to our feature set and running a new benchmark.

In [ ]:
other_nbhs = list(nbh_counts[nbh_counts <= 8].index)

X5 = X4.copy()

X5['Neighborhood'] = df['Neighborhood'].replace(other_nbhs, 'Other')

split_and_validate(pd.get_dummies(X5), y)

Great, another ~.01 R^2 boost, though at the expense of adding a large number of feature columns!

#### Getting to fancier features

Let's close out our introduction to feature engineering by considering a more complex type of feature that may work very nicely for certain problems. It doesn't seem to add a great deal over what we have so far, but it's a style of engineering to keep in mind for the future.

We'll create features that capture where a feature value lies relative to the members of a category it belongs to. In particular, we'll calculate deviance of a row's feature value from the mean value of the category that row belongs to. This helps to capture information about a feature relative to the category's distribution, e.g. how nice a house is relative to other houses in its neighborhood or of its style.

Below we define reusable code for generating features of this form, feel free to repurpose it for future feature engineering work!

In [ ]:
def add_deviation_feature(X, feature, category):
    
    # temp groupby object
    category_gb = X.groupby(category)[feature]
    
    # create columns of category means and standard deviations
    category_mean = category_gb.transform(lambda x: x.mean())
    category_std = category_gb.transform(lambda x: x.std())
    
    # compute stds from category mean for each feature value,
    # add to X as new feature
    deviation_feature = (X[feature] - category_mean) / category_std 
    X[feature + '_Dev_' + category] = deviation_feature  

And now let's use our feature generation code to add 2 new deviation features, and run a final benchmark.

In [ ]:
X6 = X5.copy()

add_deviation_feature(X6, 'Year Built', 'House Style')
add_deviation_feature(X6, 'Overall Qual', 'Neighborhood')

split_and_validate(pd.get_dummies(X6), y)

Well it didn't help much, but it did help a bit!

## Workflow Recap

**Benchmarks**:

> 1. Baseline feature set: ~.82 R^2 
> 2. Add Several polynomial transforms: ~.87 R^2
> 3. Add Several interaction terms: ~.88 R^2
> 4. Add Category features (house style): ~.89 R^2
> 5. Add Category features (neighborhood): ~.90 R^2
> 6. Add Category deviation features: ~.901 R^2

As you can see, feature engineering often follows a sort of [_Pareto principle_](https://en.wikipedia.org/wiki/Pareto_principle), where a large bulk of the predictive gains can be reached through adding a set of intuitive, strong features like polynomial transforms and interactions. Directly incorporating additional information like categorical variables can also be very helpful. Beyond this point, additional feature engineering can provide significant, but potentially diminishing returns. Whether it's worth making the extra effort is very dependent on the use case for the model! 

-----
**Further Practice Exercise**: Gamify this to practice your feature engineering skills! How far can you push the validation R^2 score with
the features we've used so far? Can you bring in new features from the original dataframe to boost the score even more? 

-----

-----
**Reflection Question**: What problems do you foresee with highly extensive feature engineering? Is the model still easy to interpret and explain? What issues might arise if we were working with a massive dataset (say 100 million+ records) instead of only a few thousand?   

-----